In [1]:
def gen_translator():
    translator = {"bn1."+str(i)+".":"bn1." for i in range(3)}

    for j in range(1,5):
        for k in range(0,2):
            for y in range(1,3):
                for i in  range(3):
                    key = "layer"+str(j)+"."+str(k)+".bn"+str(y)+"."
                    translator[key + str(i) + "."] = key

    for j in range(2,5):
        for i in range(3):
            key = "layer"+str(j)+".0."
            translator[key + "bn3." + str(i) + "."] = key + "downsample.1."
    
    return translator

# print(gen_translator())


In [3]:
import training as t
import torch
import Piggyback.networks as pbnet
import Rebuffi.networks as rbnet
import torch.utils.model_zoo as model_zoo
import copy

MODEL = "ps" # "p" or "q" or "rs" or "rp"
# Initialize the specified architecture.
if MODEL == 'p':
    model = pbnet.piggyback_net18([1,1,10], False)
elif MODEL == 'q':
    model = pbnet.quantized_net18([1,1,10], False)
elif MODEL == 'rs':
    model = rbnet.rebuffi_net18([1,1,10], serie=True, pre_imagenet=False)
elif MODEL == 'ps':
    model = rbnet.rebuffi_net18([1,1,10], serie=False, pre_imagenet=False)

dic1 = copy.deepcopy(model.state_dict())

dicz = model_zoo.load_url('https://download.pytorch.org/models/resnet18-5c106cde.pth')


#print(model.layer2[0].bn1[0].weight)
#print(model.layer2[0].conv1.weight)

model.load_state_dict(dicz, False)

dic2 = model.state_dict()

# for name in dic2.keys():
#     if not "bn" in name:
#         print(name)
# for name in dicz.keys():
#     if not "bn" in name or "downsample" in name:
#         print(name)

translator = gen_translator()

for na in dic2:
    if na[:-6] in translator:
        name = na[:-6]
        dic2[name + "weight"].copy_(dicz[translator[name] + "weight"])
        dic2[name + "bias"] = dicz[translator[name] + "bias"]

model.load_state_dict(dic2)
#print(model.layer2[0].bn1[0].weight)
#print(model.layer2[0].conv1.weight)

# print("Computing differences....")
# for name in dicz.keys():
#     if name in dic2:
#         print(name)
#         if not torch.equal(dicz[name], dic2[name]):
#             print(name + " is different")
# print(".....terminated")

for name in dic1.keys():
    # print(name)
    if torch.equal(dic1[name], dic2[name]):
        print(name + " is equal")
    else:
        print(name + "is different")
print(".....terminated")


Model pretrained loaded
conv1.weightis different
bn1.0.weightis different
bn1.0.biasis different
bn1.0.running_mean is equal
bn1.0.running_var is equal
bn1.1.weightis different
bn1.1.biasis different
bn1.1.running_mean is equal
bn1.1.running_var is equal
bn1.2.weightis different
bn1.2.biasis different
bn1.2.running_mean is equal
bn1.2.running_var is equal
layer1.0.conv1.weightis different
layer1.0.alfa1.conv.0.weight is equal
layer1.0.alfa1.conv.1.weight is equal
layer1.0.alfa1.conv.2.weight is equal
layer1.0.bn1.0.weightis different
layer1.0.bn1.0.biasis different
layer1.0.bn1.0.running_mean is equal
layer1.0.bn1.0.running_var is equal
layer1.0.bn1.1.weightis different
layer1.0.bn1.1.biasis different
layer1.0.bn1.1.running_mean is equal
layer1.0.bn1.1.running_var is equal
layer1.0.bn1.2.weightis different
layer1.0.bn1.2.biasis different
layer1.0.bn1.2.running_mean is equal
layer1.0.bn1.2.running_var is equal
layer1.0.conv2.weightis different
layer1.0.alfa2.conv.0.weight is equal
layer

In [1]:
# Check if the code is correctly loading the parameters.
import training as t
import torch
import Piggyback.networks as pbnet
import Rebuffi.networks as rbnet
import torch.utils.model_zoo as model_zoo
import copy

MODEL = "ps" # "p" or "q" or "rs" or "rp"
# Initialize the specified architecture.
if MODEL == 'p':
    model = pbnet.piggyback_net18([1,1,10], True)
elif MODEL == 'q':
    model = pbnet.quantized_net18([1,1,10], True)
elif MODEL == 'rs':
    model = rbnet.rebuffi_net18([1,1,10], serie=True, pre_imagenet=True)
elif MODEL == 'ps':
    model = rbnet.rebuffi_net18([1,1,10], serie=False, pre_imagenet=True)
else:
    raise NotImplementedError

dic1 = model.state_dict()
dicz = model_zoo.load_url('https://download.pytorch.org/models/resnet18-5c106cde.pth')

name = "conv1.weight"
if not torch.equal(dic1[name], dicz[name]):
    print("not equal")
else:
    print("equal")
    
if not torch.equal(dic1["layer1.0.bn1.2.weight"], dicz["layer1.0.bn1.weight"]):
    print("not equal")
else:
    print("equal")

Model pretrained loaded
equal
equal


In [5]:
# Check if the code is correctly setting requires_grad=True or False
import training as t
import torch
import Piggyback.networks as pbnet
import Rebuffi.networks as rbnet
import torch.utils.model_zoo as model_zoo
import copy

MODEL = "p" # "p" or "q" or "rs" or "rp"
# Initialize the specified architecture.
if MODEL == 'p':
    model = pbnet.piggyback_net18([1,1,10], True)
elif MODEL == 'q':
    model = pbnet.quantized_net18([1,1,10], True)
elif MODEL == 'rs':
    model = rbnet.rebuffi_net18([1,1,10], serie=True, pre_imagenet=True)
elif MODEL == 'ps':
    model = rbnet.rebuffi_net18([1,1,10], serie=False, pre_imagenet=True)
else:
    raise NotImplementedError
    
model.set_index(1)

# Consider this as a sanity check
for name, par in model.named_parameters():
    if par.requires_grad:
        print(name + " requires_grad: " + str(par.requires_grad))
        


Model pretrained loaded
conv1.mask.1 requires_grad: True
bn1.1.weight requires_grad: True
bn1.1.bias requires_grad: True
layer1.0.conv1.mask.1 requires_grad: True
layer1.0.bn1.1.weight requires_grad: True
layer1.0.bn1.1.bias requires_grad: True
layer1.0.conv2.mask.1 requires_grad: True
layer1.0.bn2.1.weight requires_grad: True
layer1.0.bn2.1.bias requires_grad: True
layer1.0.downsample.0.mask.1 requires_grad: True
layer1.0.bn3.1.weight requires_grad: True
layer1.0.bn3.1.bias requires_grad: True
layer1.1.conv1.mask.1 requires_grad: True
layer1.1.bn1.1.weight requires_grad: True
layer1.1.bn1.1.bias requires_grad: True
layer1.1.conv2.mask.1 requires_grad: True
layer1.1.bn2.1.weight requires_grad: True
layer1.1.bn2.1.bias requires_grad: True
layer2.0.conv1.mask.1 requires_grad: True
layer2.0.bn1.1.weight requires_grad: True
layer2.0.bn1.1.bias requires_grad: True
layer2.0.conv2.mask.1 requires_grad: True
layer2.0.bn2.1.weight requires_grad: True
layer2.0.bn2.1.bias requires_grad: True
laye